In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
google_api_key = os.getenv('GOOGLE_API_KEY')
if not google_api_key: 
    raise ValueError("GOOGLE_API_KEY is not set in the environment variables.")

In [2]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [3]:
from langgraph.prebuilt import create_react_agent

def get_weather(city: str) -> str:  
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_react_agent(
    model=llm,  
    tools=[get_weather],  
    prompt="You are a helpful assistant"  
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in San Francisco?"}]}
)

{'messages': [HumanMessage(content='what is the weather in San Francisco?', additional_kwargs={}, response_metadata={}, id='8d685988-d31a-40ef-8933-583be44ed3cf'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "San Francisco"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--983de77a-32a7-45bf-8c08-995ad1d25224-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'b03141c8-c6a3-48c9-a355-2ef6113c0aaa', 'type': 'tool_call'}], usage_metadata={'input_tokens': 27, 'output_tokens': 6, 'total_tokens': 33, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content="It's always sunny in San Francisco!", name='get_weather', id='2d9f0b5f-09a6-4d99-87c4-9a2735f0375b', tool_call_id='b03141c8-c6a3-48c9-a355-2ef6113c0aaa'),
  AIMessage(content="It's always sunny in San Francisco!"

In [4]:
agent = create_react_agent(
    model=llm,  
    tools=[get_weather],  
    prompt="Never answer questions about the weather."  
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in San Francisco?"}]}
)

{'messages': [HumanMessage(content='what is the weather in San Francisco?', additional_kwargs={}, response_metadata={}, id='e8e4aa43-9fa0-4a68-82f1-f93f70e9934a'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "San Francisco"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--8b5b4749-db3f-4a1f-ae1e-fb6b156014b3-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': '27e4fca1-b1fc-4485-b138-0009d0c83430', 'type': 'tool_call'}], usage_metadata={'input_tokens': 29, 'output_tokens': 6, 'total_tokens': 35, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content="It's always sunny in San Francisco!", name='get_weather', id='33e20686-8a6f-4b90-b56a-5d35eb1c427a', tool_call_id='27e4fca1-b1fc-4485-b138-0009d0c83430'),
  AIMessage(content="It's always sunny in San Francisco!"

In [5]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

agent = create_react_agent(
    model=llm,
    tools=[get_weather],
    checkpointer=checkpointer  
)

# Run the agent
config = {"configurable": {"thread_id": "1"}}
sf_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in san francisco?"}]},
    config  
)
ny_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what about new york?"}]},
    config
)

In [6]:
sf_response, ny_response

({'messages': [HumanMessage(content='what is the weather in san francisco?', additional_kwargs={}, response_metadata={}, id='b0c89cd6-21a1-491d-9504-9a6278a45995'),
   AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "san francisco"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--8f0bbb22-7dee-4f7d-98a6-1aa5157913b1-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'san francisco'}, 'id': 'a403af1d-c4ca-4660-8b02-10817d467db8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 22, 'output_tokens': 6, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}}),
   ToolMessage(content="It's always sunny in san francisco!", name='get_weather', id='43d7076d-a8bc-491c-bb93-8cc1fa6ab434', tool_call_id='a403af1d-c4ca-4660-8b02-10817d467db8'),
   AIMessage(content="It's always sunny in san francis

In [7]:
from pydantic import BaseModel

class WeatherResponse(BaseModel):
    city: str
    conditions: str

agent = create_react_agent(
    model=llm,
    tools=[get_weather],
    response_format=WeatherResponse  
)
response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in san francisco?"}]}
)

response["structured_response"]

WeatherResponse(city='San Francisco', conditions='Sunny')